# L'opérateur LEFT JOIN
---

Cours : SQL pour Débutant(e)s Jour 4 -Jointures, Sous-Requêtes & Vues
Lien : https://www.youtube.com/watch?v=g9yC8pcdRFc&list=PLyh35eYRez8cAEBfi1LWN70Cs77bBvCOp&index=4

Github le coin stat : https://github.com/LeCoinStat/LeCoinStat/tree/main/Bien_Debuter_SQL
Fichiers Github récupérés : https://github.com/LeCoinStat/LeCoinStat/tree/main/Bien_Debuter_SQL/Jour4

[Les différentes jointures](https://sql.sh/2401-sql-join-infographie)

In [1]:
import duckdb
from skimpy import skim

In [2]:
# Connexion à la librairie duckdb
conn = duckdb.connect()

## Fichier des ventes

In [3]:
# Requête SQL : récupération de toutes les données du fichier .csv en DF
sales_df = conn.execute(
    """
    SELECT *
    FROM read_csv_auto('data/indexE/Ventes.csv', header=True)
    """
).df()

In [4]:
# Stats de la DF
skim(sales_df)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 100    │ │ int32       │ 5     │                                                          │
│ │ Number of columns │ 7      │ │ datetime64  │ 1     │                                                          │
│ └───────────────────┴────────┘ │ float64     │ 1     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┓  │
│ ┃ column_name        ┃ NA  ┃ NA %   ┃ mean    ┃ sd     ┃ p0     ┃ p25    ┃ p50   ┃ p75    ┃ p100   ┃ hist    ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━┩  │
│ │ VenteID            │   0 │      0 │    50.5 │  29.01 │      1 │  25.75 │  50.5 │  75.25 │    100 │ ▇▇▇▇▇▇  │  │
│ │ ClientID           │   0 │      0 │   51.26 │  29.18 │      1 │  27.75 │  53.5 │  77.25 │     99 │ ▆▇▅▇▆▇  │  │
│ │ EmployeID          │   0 │      0 │   50.65 │   29.4 │      2 │     23 │  49.5 │  76.25 │    100 │ ▇▇▇▆▇▇  │  │
│ │ ProduitID          │   0 │      0 │   48.08 │  24.56 │      3 │     29 │    46 │  65.25 │     98 │ ▅▆▇▇▅▃  │  │
│ │ QuantiteVendue     │   0 │      0 │   48.36 │  30.17 │      1 │     22 │  50.5 │     71 │    100 │ ▇▇▃▇▃▇  │  │
│ │ MontantTotal       │   0 │      0 │    8278 │  11580 │  59.98 │   1542 │  2979 │   9249 │  48500 │ ▇▁▁  ▁  │  │
│ └────────────────────┴─────┴────────┴─────────┴────────┴────────┴────────┴───────┴────────┴────────┴─────────┘  │
│                                                    datetime                                                     │
│ ┏━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓  │
│ ┃ column_name      ┃ NA   ┃ NA %    ┃ first                      ┃ last                       ┃ frequency    ┃  │
│ ┡━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩  │
│ │ DateVente        │    0 │       0 │    2020-01-08 12:43:33     │    2023-12-27 15:39:42     │ None         │  │
│ └──────────────────┴──────┴─────────┴────────────────────────────┴────────────────────────────┴──────────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯

In [5]:
# Enregistrement de la DF ci-avant en BD virtuelle (requête SQL : CTE)
conn.register("sales_db", sales_df)

## Fichier des employés

In [7]:
# Requête SQL : récupération de toutes les données du fichier .csv en DF
employees_df = conn.execute(
    """
    SELECT *
    FROM read_csv_auto('data/indexE/Employes.csv', header=True)
    """
).df()

In [8]:
# Stats de la DF
skim(employees_df)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 100    │ │ string      │ 5     │                                                          │
│ │ Number of columns │ 6      │ │ int32       │ 1     │                                                          │
│ └───────────────────┴────────┘ └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━┳━━━━━━━━━┓  │
│ ┃ column_name      ┃ NA   ┃ NA %   ┃ mean   ┃ sd       ┃ p0   ┃ p25     ┃ p50    ┃ p75     ┃ p100  ┃ hist    ┃  │
│ ┡━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━╇━━━━━━━━━┩  │
│ │ EmployeID        │    0 │      0 │   50.5 │    29.01 │    1 │   25.75 │   50.5 │   75.25 │   100 │ ▇▇▇▇▇▇  │  │
│ └──────────────────┴──────┴────────┴────────┴──────────┴──────┴─────────┴────────┴─────────┴───────┴─────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name                     ┃ NA     ┃ NA %       ┃ words per row              ┃ total words           ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ Nom                             │      0 │          0 │                          1 │                   100 │  │
│ │ Prenom                          │      0 │          0 │                          1 │                   100 │  │
│ │ Fonction                        │      0 │          0 │                        2.1 │                   207 │  │
│ │ Email                           │      0 │          0 │                          1 │                   100 │  │
│ │ NumeroTelephone                 │      0 │          0 │                          1 │                   100 │  │
│ └─────────────────────────────────┴────────┴────────────┴────────────────────────────┴───────────────────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯

In [9]:
# Enregistrement de la DF ci-avant en BD virtuelle (requête SQL : CTE)
conn.register("employees_db", employees_df)

## Fichier des fournisseurs

In [19]:
# Requête SQL : récupération de toutes les données du fichier .csv en DF
suppliers_df = conn.execute(
    """
    SELECT *
    FROM read_csv_auto('data/indexE/Fournisseurs.csv', header=True)
    """
).df()

In [20]:
# Stats de la DF
skim(suppliers_df)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 100    │ │ string      │ 4     │                                                          │
│ │ Number of columns │ 5      │ │ int32       │ 1     │                                                          │
│ └───────────────────┴────────┘ └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━┳━━━━━━━━━┓  │
│ ┃ column_name        ┃ NA   ┃ NA %   ┃ mean   ┃ sd      ┃ p0  ┃ p25     ┃ p50    ┃ p75     ┃ p100  ┃ hist    ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━╇━━━━━━━━━┩  │
│ │ FournisseurID      │    0 │      0 │   50.5 │   29.01 │   1 │   25.75 │   50.5 │   75.25 │   100 │ ▇▇▇▇▇▇  │  │
│ └────────────────────┴──────┴────────┴────────┴─────────┴─────┴─────────┴────────┴─────────┴───────┴─────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name                     ┃ NA     ┃ NA %       ┃ words per row              ┃ total words           ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ NomFournisseur                  │      0 │          0 │                        2.3 │                   227 │  │
│ │ Adresse                         │      0 │          0 │                        7.4 │                   742 │  │
│ │ Email                           │      0 │          0 │                          1 │                   100 │  │
│ │ NumeroTelephone                 │      0 │          0 │                          1 │                   100 │  │
│ └─────────────────────────────────┴────────┴────────────┴────────────────────────────┴───────────────────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯

In [21]:
# Enregistrement de la DF ci-avant en BD virtuelle (requête SQL : CTE)
conn.register("suppliers_db", suppliers_df)

## Fichier des produits

In [13]:
# Requête SQL : récupération de toutes les données du fichier .csv en DF
products_df = conn.execute(
    """
    SELECT *
    FROM read_csv_auto('data/indexE/Produits.csv', header=True)
    """
).df()

In [14]:
# Stats de la DF
skim(products_df)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 100    │ │ int32       │ 3     │                                                          │
│ │ Number of columns │ 5      │ │ string      │ 2     │                                                          │
│ └───────────────────┴────────┘ └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━┓  │
│ ┃ column_name        ┃ NA   ┃ NA %   ┃ mean    ┃ sd      ┃ p0  ┃ p25     ┃ p50    ┃ p75    ┃ p100  ┃ hist    ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━┩  │
│ │ ProduitID          │    0 │      0 │    50.5 │   29.01 │   1 │   25.75 │   50.5 │  75.25 │   100 │ ▇▇▇▇▇▇  │  │
│ │ PrixUnitaire       │    0 │      0 │   486.7 │   326.6 │  59 │     129 │    399 │    699 │   999 │ ▇▃▂ ▇▃  │  │
│ │ FournisseurID      │    0 │      0 │   54.04 │   30.51 │   1 │      31 │     56 │     84 │   100 │ ▆▃▅▃▅▇  │  │
│ └────────────────────┴──────┴────────┴─────────┴─────────┴─────┴─────────┴────────┴────────┴───────┴─────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name               ┃ NA      ┃ NA %       ┃ words per row                ┃ total words              ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ NomProduit                │       0 │          0 │                          2.7 │                      266 │  │
│ │ Description               │       0 │          0 │                           14 │                     1371 │  │
│ └───────────────────────────┴─────────┴────────────┴──────────────────────────────┴──────────────────────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯

In [15]:
# Enregistrement de la DF ci-avant en BD virtuelle (requête SQL : CTE)
conn.register("products_db", products_df)

## Requêtes SQL

Donner pour chaque employé, le nom, le prénom et le nombre de ventes réalisées
et il faut conserver les employés qui n'ont réalisé aucune vente

In [18]:
conn.execute(
    """
    -- Champs à récupérer
    SELECT
        e.Nom
        , e.Prenom
        , COUNT(s.VenteID) AS "Nombre de ventes"
    -- Jointure
    FROM employees_db e
        LEFT JOIN sales_db s
        ON e.EmployeID = s.EmployeID
    -- TCD
    GROUP BY
        e.Nom
        , e.Prenom
    -- Trie
    ORDER BY 
        "Nombre de ventes" DESC
        , e.Nom ASC
        , e.Prenom ASC
    """
).df()

,Nom,Prenom,Nombre de ventes
0,Ball,Alyssa,3
1,Edwards,Kirsten,3
2,Gordon,Gabriella,3
3,Graham,Mr.,3
4,Liu,Timothy,3
...,...,...,...
95,Thomas,Gregory,0
96,Torres,Fernando,0
97,Tran,Michael,0
98,West,Natasha,0


Donner pour chaque fournisseur, son nom, son email et le nombre de produits fournis et conserver les fournisseurs qui ne fournisseurs aucun produit

In [22]:
conn.execute(
    """
    -- Champs à récupérer
    SELECT
        s.NomFournisseur AS Nom
        , s.Email
        , COUNT(p.ProduitID) AS "Nombre de produits fournis"
    -- Jointure
    FROM suppliers_db s
        LEFT JOIN products_db p
        ON s.FournisseurID = p.FournisseurID
    -- TCD
    GROUP BY
        Nom
        , s.Email
    -- Trie
    ORDER BY
        "Nombre de produits fournis" DESC
        , Nom ASC
    """
).df()

,Nom,Email,Nombre de produits fournis
0,Velasquez-Irwin,velasquez-irwin@gmail.com,6
1,Barnett-Davidson,barnett-davidson@gmail.com,3
2,Bridges Ltd,bridgesltd@gmail.com,3
3,Brown-Walker,brown-walker@gmail.com,3
4,Gaines and Sons,gainesandsons@gmail.com,3
...,...,...,...
95,"Thomas, Valdez and Escobar",thomas_valdezandescobar@gmail.com,0
96,Thornton Ltd,thorntonltd@gmail.com,0
97,Vasquez-Jennings,vasquez-jennings@gmail.com,0
98,Washington PLC,washingtonplc@gmail.com,0
